In [2]:
import pandas as pd 
import os
import numpy as np

In [69]:
census_files = ['../assets/census/customVariables/'+i \
                for i in os.listdir('../assets/census/customVariables/')]
census_files

['../assets/census/customVariables/ACS5Y_2011_B02001.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2011_B17020.csv',
 '../assets/census/customVariables/ACS5Y_2011_B19013.csv',
 '../assets/census/customVariables/ACS5Y_2016_B02001.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2016_B17020.csv',
 '../assets/census/customVariables/ACS5Y_2016_B19013.csv',
 '../assets/census/customVariables/ACS5Y_2021_B02001.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2021_B170

In [4]:
def sort_by_year(file_name):

    year_part = file_name.split("_")[1]

    return year_part

In [86]:
var_to_interpolate = 'B15002F'
census_files_to_interp = sorted([i for i in census_files if var_to_interpolate in i ], key=sort_by_year)
census_files_to_interp

['../assets/census/customVariables/DEC_2000_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002F.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002F.csv']

In [74]:
df = pd.read_csv(census_files_to_interp[0])
df

,B15002_002E,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_011E,B15002_012E,B15002_013E,B15002_014E,B15002_015E,B15002_016E,B15002_017E,B15002_018E,PAT_ZIP
0,570,5,14,29,35,22,19,4,16,154,53,81,13,109,16,0,0,73949
1,3350,14,3,20,10,9,0,21,24,412,141,550,107,1282,532,170,55,75001
2,11112,59,23,55,48,59,81,72,159,1548,602,2205,993,3879,1094,109,126,75002
3,14499,283,257,817,406,473,296,300,498,2487,771,2225,904,3452,910,279,141,75006
4,15470,111,56,96,97,86,234,130,401,2307,1029,2669,1046,5140,1511,353,204,75007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,23325,289,132,685,382,564,289,381,895,5152,2149,5644,1673,3455,1048,469,118,79936
1832,4869,132,109,600,240,302,296,170,203,1368,350,572,208,230,55,30,4,79938
1833,2219,290,185,473,145,166,80,72,104,424,60,150,20,44,0,0,6,88063
1834,10025,268,72,137,366,244,280,423,388,3471,784,1537,454,987,448,126,40,88220


In [75]:
def div_by_population(column, total_pop):
    return column / df[total_pop]

In [88]:
def processEducationData(base_df, male=True):
    if male:
        total_pop_column = 'B15002_002E'
    else:
        total_pop_column = 'B15002_019E'
    df = base_df.drop('PAT_ZIP',axis=1)

    derived_df = df.drop(total_pop_column, axis=1).apply(lambda x: div_by_population(x, total_pop_column))
    derived_df.insert(0, 'PAT_ZIP', base_df['PAT_ZIP'])

    return derived_df

In [81]:
def processPovertyData(base_df):
    df = base_df.drop('PAT_ZIP',axis=1)
    derived_df = pd.DataFrame()

    var_names = sorted(list(set([i.split('_')[0] for i in df.columns if 'PAT_ZIP' not in i])))

    for ind, key in enumerate(var_names):
        # print(key)
        temp_df = df.filter(like=key, axis=1)
        # print(temp_df.filter(like='002',axis=1))
        divided = temp_df.filter(like='002', axis=1).values/temp_df.filter(like='001', axis=1).values
        divided = divided.ravel()
        # print(divided.shape)
        derived_df.insert(ind, key, divided)
        # print(temp_df.columns)
    derived_df.insert(0, 'PAT_ZIP', base_df['PAT_ZIP'])
    return derived_df


In [82]:
census_files_to_interp

['../assets/census/customVariables/DEC_2000_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002M.csv']

In [89]:
for file in census_files_to_interp:
    df = pd.read_csv(file)
    if 'state' in df.columns:
        df = df.drop('state', axis=1)
    
    # cleaned_df = processPovertyData(df)
    cleaned_df = processEducationData(df, male=False)

    # cleaned_df = df.drop(['B02001_001E','PAT_ZIP'],axis=1).apply(divide_by_total)
    # don't need to normalize for B19013
    # cleaned_df = df.drop(['PAT_ZIP'],axis=1)
    # cleaned_df.insert(0,'PAT_ZIP',df['PAT_ZIP'])
    # cleaned_df = cleaned_df.dropna()
    cleaned_df.to_csv(f'../assets/census/derivedVariables/{sort_by_year(file)}_{var_to_interpolate}.csv',index=False)
    
    

In [95]:
file_path = "../assets/census/derivedVariables/"
B15_files = [file_path+i for i in os.listdir(file_path) if '.csv' in i]

In [107]:
B15_files

['../assets/census/derivedVariables/2000_B15002F.csv',
 '../assets/census/derivedVariables/2000_B15002M.csv',
 '../assets/census/derivedVariables/2011_B15002F.csv',
 '../assets/census/derivedVariables/2011_B15002M.csv',
 '../assets/census/derivedVariables/2016_B15002F.csv',
 '../assets/census/derivedVariables/2016_B15002M.csv',
 '../assets/census/derivedVariables/2021_B15002F.csv',
 '../assets/census/derivedVariables/2021_B15002M.csv']

In [115]:
for i in range(4):
    # print(i*2, i*2+2)

    df1_path, df2_path = B15_files[i*2:i*2+2]
    df1 = pd.read_csv(df1_path)
    df2 = pd.read_csv(df2_path)

    df = df1.merge(df2, on='PAT_ZIP')
    df.to_csv(f'{df1_path[:-4]}.csv',index=False)

In [63]:
df = pd.read_csv(census_files_to_interp[0])
df

,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,state,PAT_ZIP
0,12510,9170,1444,34,686,136,891,149,48,75001
1,62175,46812,6156,90,5272,0,1470,2375,48,75002
2,45931,35798,3461,328,3206,0,2525,613,48,75006
3,50921,35589,4542,593,6820,30,1634,1713,48,75007
4,8418,7387,112,207,0,0,505,207,48,75009
...,...,...,...,...,...,...,...,...,...,...
1930,17995,12349,2694,9,332,16,1481,1114,48,79934
1931,18249,14543,584,89,98,22,2527,386,48,79935
1932,114941,96682,2237,576,737,0,12809,1900,48,79936
1933,51367,42417,1381,437,401,72,5320,1339,48,79938


In [7]:
def divide_by_total(column, population_column):
    return column/df['B02001_001E']

In [74]:
cleaned_df = df.drop(['B02001_001E','state','PAT_ZIP'],axis=1).apply(divide_by_total)
cleaned_df.insert(0,'PAT_ZIP',df['PAT_ZIP'])
cleaned_df

,PAT_ZIP,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E
0,75001,0.733014,0.115428,0.002718,0.054836,0.010871,0.071223,0.011910
1,75002,0.752907,0.099011,0.001448,0.084793,0.000000,0.023643,0.038199
2,75006,0.779386,0.075352,0.007141,0.069800,0.000000,0.054974,0.013346
3,75007,0.698906,0.089197,0.011645,0.133933,0.000589,0.032089,0.033640
4,75009,0.877524,0.013305,0.024590,0.000000,0.000000,0.059990,0.024590
...,...,...,...,...,...,...,...,...
1930,79934,0.686246,0.149708,0.000500,0.018450,0.000889,0.082301,0.061906
1931,79935,0.796920,0.032002,0.004877,0.005370,0.001206,0.138473,0.021152
1932,79936,0.841145,0.019462,0.005011,0.006412,0.000000,0.111440,0.016530
1933,79938,0.825764,0.026885,0.008507,0.007807,0.001402,0.103568,0.026067


In [75]:
cleaned_df.to_csv('../assets/census/derivedVariables/2011_B02001.csv',index=False)

In [48]:
import cenpy
dataset = 'ACSDT5Y2011'
conn = cenpy.remote.APIConnection(dataset)

In [49]:
conn_df = conn.variables

In [68]:
for i in range(0,8):
      print(conn_df.loc[df.columns[i]]['label'], '\n', 
            conn_df.loc[df.columns[i]]['concept'])

Estimate!!Total 
 RACE
Estimate!!Total!!White alone 
 RACE
Estimate!!Total!!Black or African American alone 
 RACE
Estimate!!Total!!American Indian and Alaska Native alone 
 RACE
Estimate!!Total!!Asian alone 
 RACE
Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone 
 RACE
Estimate!!Total!!Some other race alone 
 RACE
Estimate!!Total!!Two or more races 
 RACE
